In [12]:
from dataclasses import dataclass

@dataclass
class Order:
    order_id: int
    side: str      # 'buy' or 'sell'
    price: float
    qty: int
    timestamp: int


In [13]:
import heapq

class OrderBook:
    def __init__(self):
        self.bids = []   # max heap (negative prices)
        self.asks = []   # min heap
        self.trades = []
        self.time = 0    # discrete event clock


    def submit(self, side, price, qty):
        self.time += 1
        order = Order(
            order_id=self.time,
            side=side,
            price=price,
            qty=qty,
            timestamp=self.time
        )

        if side == "buy":
            heapq.heappush(self.bids, (-price, order.timestamp, order))
        else:
            heapq.heappush(self.asks, (price, order.timestamp, order))

        self.match()


    def match(self):
        while self.bids and self.asks:
            best_bid_price, _, bid = self.bids[0]
            best_ask_price, _, ask = self.asks[0]

            best_bid_price = -best_bid_price

            if best_bid_price < best_ask_price:
                break

            # Execute trade
            trade_qty = min(bid.qty, ask.qty)

            self.trades.append({
                "price": best_ask_price,
                "qty": trade_qty,
                "buy_id": bid.order_id,
                "sell_id": ask.order_id
            })

            bid.qty -= trade_qty
            ask.qty -= trade_qty

            if bid.qty == 0:
                heapq.heappop(self.bids)
            if ask.qty == 0:
                heapq.heappop(self.asks)


    def print_top_5(self):
        print("\n--- ORDER BOOK ---")

        top_bids = sorted(self.bids, reverse=True)[:5]
        top_asks = sorted(self.asks)[:5]

        print("BIDS:")
        for p, _, o in top_bids:
            print(f"{-p} | qty={o.qty}")

        print("ASKS:")
        for p, _, o in top_asks:
            print(f"{p} | qty={o.qty}")


In [14]:
ob = OrderBook()

orders = [
    ("buy", 100, 10),
    ("buy", 101, 5),
    ("sell", 102, 7),
    ("sell", 100, 4),
    ("buy", 99, 8),
]

for o in orders:
    ob.submit(*o)
    ob.print_top_5()



--- ORDER BOOK ---
BIDS:
100 | qty=10
ASKS:

--- ORDER BOOK ---
BIDS:
100 | qty=10
101 | qty=5
ASKS:

--- ORDER BOOK ---
BIDS:
100 | qty=10
101 | qty=5
ASKS:
102 | qty=7

--- ORDER BOOK ---
BIDS:
100 | qty=10
101 | qty=1
ASKS:
102 | qty=7

--- ORDER BOOK ---
BIDS:
99 | qty=8
100 | qty=10
101 | qty=1
ASKS:
102 | qty=7


In [15]:
def run_stream(order_stream):
    ob = OrderBook()
    for o in order_stream:
        ob.submit(*o)
    return ob.trades

t1 = run_stream(orders)
t2 = run_stream(orders)

assert t1 == t2
print("✅ Deterministic replay confirmed")


✅ Deterministic replay confirmed
